In [6]:
import scipy.stats as stats
import numpy as np

# Исходные параметры
baseline_conversion = 0.1  # Текущий процент плательщиков
expected_conversion = 0.11  # Ожидаемый процент плательщиков
alpha = 0.05  # Уровень значимости
power = 0.80  # Мощность
daily_new_players = 100  # Количество новых игроков в день

# Расчет стандартного отклонения для двух пропорций
std_dev = np.sqrt(baseline_conversion * (1 - baseline_conversion) +
                  expected_conversion * (1 - expected_conversion))

# Расчет размера выборки для каждой группы
effect_size = (stats.norm.ppf(1 - alpha/2) + stats.norm.ppf(power)) * std_dev
sample_size_per_group = (effect_size**2) / ((expected_conversion - baseline_conversion)**2)

# Расчет общего размера выборки и длительности эксперимента
total_sample_size = sample_size_per_group * 2  # Для двух групп
experiment_duration = total_sample_size / daily_new_players

print(f"Sample Size per Group: {sample_size_per_group}")
print(f"Total Sample Size: {total_sample_size}")
print(f"Experiment Duration (days): {experiment_duration}")

Sample Size per Group: 14748.045020841952
Total Sample Size: 29496.090041683903
Experiment Duration (days): 294.960900416839


In [9]:
from scipy.stats import norm, ttest_ind, bayes_mvs

# Установка сида для воспроизводимости
np.random.seed(0)

# Генерация контрольной группы
control_group_size = 15000
control_conversion_rate = 0.1
control_group = np.random.binomial(1, control_conversion_rate, control_group_size)

# Генерация экспериментальных групп
experimental_group_sizes = [15000, 15000, 15000]
experimental_conversion_rates = [0.09, 0.11, 0.1]  # Хуже, лучше, без эффекта
experimental_groups = [np.random.binomial(1, rate, size) for rate, size in zip(experimental_conversion_rates, experimental_group_sizes)]

# Расчет средних и стандартных отклонений
control_mean = np.mean(control_group)
control_std = np.std(control_group, ddof=1)

# Расчет доверительных интервалов (Frequentist и Bayesian) для контрольной группы
control_conf_interval_freq = norm.interval(0.95, loc=control_mean, scale=control_std/np.sqrt(control_group_size))
control_conf_interval_bayes = bayes_mvs(control_group, alpha=0.95)[0][1]

# Анализ экспериментальных групп
experimental_results = []

for i, exp_group in enumerate(experimental_groups):
    exp_mean = np.mean(exp_group)
    exp_std = np.std(exp_group, ddof=1)
    exp_size = experimental_group_sizes[i]

    # Frequentist доверительный интервал
    exp_conf_interval_freq = norm.interval(0.95, loc=exp_mean, scale=exp_std/np.sqrt(exp_size))
    
    # Bayesian доверительный интервал
    exp_conf_interval_bayes = bayes_mvs(exp_group, alpha=0.95)[0][1]

    # T-тест
    t_stat, p_value = ttest_ind(control_group, exp_group)

    # Расчет эффекта
    effect_size = exp_mean - control_mean

    experimental_results.append({
        "Experiment": i+1,
        "Mean": exp_mean,
        "Frequentist 95% CI": exp_conf_interval_freq,
        "Bayesian 95% CI": exp_conf_interval_bayes,
        "T-Statistic": t_stat,
        "P-Value": p_value,
        "Effect Size": effect_size
    })

# Вывод результатов
(control_mean, control_conf_interval_freq, control_conf_interval_bayes), experimental_results



((0.09853333333333333,
  (0.093763717919026, 0.10330294874764066),
  (0.09376387690885635, 0.10330278975781032)),
 [{'Experiment': 1,
   'Mean': 0.0928,
   'Frequentist 95% CI': (0.08815653210323636, 0.09744346789676363),
   'Bayesian 95% CI': (0.08815668688807939, 0.0974433131119206),
   'T-Statistic': 1.6881047176927513,
   'P-Value': 0.09140152364159299,
   'Effect Size': -0.00573333333333334},
  {'Experiment': 2,
   'Mean': 0.1086,
   'Frequentist 95% CI': (0.10362070145724356, 0.11357929854275645),
   'Bayesian 95% CI': (0.10362086743662802, 0.11357913256337199),
   'T-Statistic': -2.8614891833740193,
   'P-Value': 0.004219474014611251,
   'Effect Size': 0.010066666666666668},
  {'Experiment': 3,
   'Mean': 0.09546666666666667,
   'Frequentist 95% CI': (0.09076388186654444, 0.1001694514667889),
   'Bayesian 95% CI': (0.09076403862865053, 0.10016929470468282),
   'T-Statistic': 0.8973436911080782,
   'P-Value': 0.3695427508358543,
   'Effect Size': -0.003066666666666662}])